In [2]:
:dep ../
use music_theory::prelude::*;
use AccidentalSign as Accidental;
use std::num::{NonZeroU16, NonZeroI16};
use music_theory::scales::{heptatonic::*, numeral::*, sized_scale::*, typed_scale::*, dyn_scale::*, pentatonic::*, rooted::*};

## Pitches
Pitches represent a letter + accidental, aware of spelling. If you don't want enharmonic spelling, use `PitchClass`.

In [ ]:
// Create pitches from letter & accidental,
let c_sharp = Pitch::from_letter_and_accidental(Letter::C, Accidental::SHARP);
// .. constant
let a_double_sharp = Pitch::A_DOUBLE_SHARP;
// or string (&str)
let d_flat = "Db".parse::<Pitch>().unwrap();

// An arbitrary amount of sharps / flats are allowed*
// (see https://dinoslice.com/posts/representing-pitches/ for more info) 
let sharps = AccidentalSign { offset: 4500 };
let very_sharp_b = Pitch::from_letter_and_accidental(Letter::B, sharps);

dbg!(c_sharp, a_double_sharp, d_flat, very_sharp_b);

[src\lib.rs:117:1] c_sharp = CSharp
[src\lib.rs:117:1] a_double_sharp = ADoubleSharp
[src\lib.rs:117:1] d_flat = DFlat
[src\lib.rs:117:1] very_sharp_b = B(4500x)Sharp


Pitches can be queried for various properties, like `.letter()`, `.accidental()`, `pitch_class`, etc.

In [4]:
very_sharp_b.letter()

B

In [5]:
d_flat.accidental()

Flat

In [6]:
a_double_sharp.as_pitch_class()

B

In [7]:
d_flat.as_pitch_class()

Cs

Pitches can be transposed by intervals, and you can get the interval between pitches.

In [8]:
// You can transpose with .transpose() or the + operator
Pitch::F + Interval::DIMINISHED_SEVENTH

EDoubleFlat

In [9]:
// This is the same as Interval::between_pitches(Pitch::D, Pitch::B_FLAT)
Pitch::D.distance_to(Pitch::B_FLAT)

Interval { quality: Minor, number: IntervalNumber(6) }

You can also change how a pitch is spelled.

In [10]:
Pitch::E_SHARP.simplified()

F

In [11]:
Pitch::D_FLAT.enharmonic()

CSharp

In [12]:
// Spell a note with flats
Pitch::A_SHARP.bias(false)

BFlat

In [13]:
// ... or with sharps
Pitch::E_FLAT.bias(true)

DSharp

Pitches can also be ordered and checked for equality, ignoring spelling if desired.

In [14]:
// same "key", but spelled differently
d_flat == c_sharp

false

In [15]:
// first ordered by letter, then accidental
// (... < bb < b < natural < # < x < ...)
c_sharp < d_flat

true

In [16]:
// the EnharmonicEq trait can be used to ignore spelling
c_sharp.eq_enharmonic(&d_flat)

true

In [17]:
// the EnharmonicOrd trait can be used to order ignore spelling
// using Ord (rust trait for <, <=, ==, >, >=), this would be Fbb > E##
Pitch::F_DOUBLE_FLAT.cmp_enharmonic(&Pitch::E_DOUBLE_SHARP)

Less

## Notes
Notes are pitches with octave information. It has many of the same pitch methods, now taking into account octave.

In [18]:
let g5 = Note::new(Pitch::G, 5);

g5 + Interval::MINOR_SECOND

Note { pitch: AFlat, octave: 5 }

Notes can also be converted to & built from frequency & midi.

In [19]:
g5.as_frequency_hz()

783.99084

In [20]:
// unwrap needed, since notes can have arbitraily high/low octaves, but midi doesn't
g5.as_midi().unwrap()

79

In [21]:
// a4, which 12TET 440.0hz is tuned to
Note::from_frequency_hz(440.0).unwrap()

Note { pitch: A, octave: 4 }

In [22]:
Note::from_midi(35)

Note { pitch: B, octave: 1 }

## Interval
Intervals represent a distance between two notes.

In [ ]:
// Create intervals from quality and number
// (checked, will return None if invalid, like m5)
let m2 = Interval::new(IntervalQuality::Minor, IntervalNumber::SECOND).unwrap();

// interval quality can be Minor, Major, Perfect, or any number of Diminished/Augmented
// constants provided for dd, d, m, M, P, A, AA
// interval number has constants up to fifteenth, or can be made from nonzero
let ddddd20 = Interval::new(IntervalQuality::Diminished(NonZeroU16::new(5).unwrap()), IntervalNumber(NonZeroI16::new(20).unwrap())).unwrap();

// or use a constant
let aug4: Interval = Interval::AUGMENTED_FOURTH;

// or from string (&str)
let maj3 = "M3".parse::<Interval>().unwrap();

// to_string() used since pretty print is somewhat verbose
dbg!(m2.to_string(), ddddd20.to_string(), aug4.to_string(), maj3.to_string());

[src\lib.rs:132:1] m2.to_string() = "m2"
[src\lib.rs:132:1] ddddd20.to_string() = "ddddd20"
[src\lib.rs:132:1] aug4.to_string() = "A4"
[src\lib.rs:132:1] maj3.to_string() = "M3"


Intervals can be simplified and inverted. You can also check how many semitones an interval spans.

In [24]:
Interval::MAJOR_THIRTEENTH.as_simple().to_string()

"M6"

In [25]:
Interval::PERFECT_FIFTH.inverted().to_string()

"P4"

In [26]:
Interval::MAJOR_SIXTH.semitones()

Semitone(9)

In [27]:
// descending intervals semitones value is negative
(-Interval::MINOR_SEVENTH).semitones()

Semitone(-10)

You can check for subzero intervals, and expand them if they are. Subzero intervals are intervals whose direction is opposite from the direction the note is moved.

In [28]:
let d1 = Interval::new(IntervalQuality::DIMINISHED, IntervalNumber::UNISON).unwrap();

d1.is_subzero()

true

In [29]:
// after transposing, note is one semitone lower, despite the interval being ascending
Note::new(Pitch::C, 4) + d1

Note { pitch: CFlat, octave: 4 }

In [30]:
// expanding a subzero interval octaves so the interval isn't subzero anymore
d1.expand_subzero().to_string()

"d8"

Intervals also have direction.

In [31]:
let c4 = Note::MIDDLE_C;

let negative_d5 = -Interval::DIMINISHED_FIFTH;

// adding a negative interval will move the note down
c4 + negative_d5

Note { pitch: FSharp, octave: 3 }

In [32]:
// an abs() method is also provided
c4 + negative_d5.abs()

Note { pitch: GFlat, octave: 4 }

Interval also supports complex intervals.

In [33]:
c4 + Interval::MAJOR_FOURTEENTH

Note { pitch: B, octave: 5 }

In [34]:
// You can make them simple again
c4 + Interval::MAJOR_FOURTEENTH.as_simple()

Note { pitch: B, octave: 4 }

Adding intervals is also possible, and it can compose two transpositions together.

In [35]:
let p5 = Interval::PERFECT_FIFTH;

let a3 = Interval::AUGMENTED_THIRD;

(c4 + p5) + a3

Note { pitch: BSharp, octave: 4 }

In [36]:
let sum = p5 + a3;

sum.to_string()

"A7"

In [37]:
c4 + sum

Note { pitch: BSharp, octave: 4 }

## Key
Keys are collections of pitches organized around a central note, following the pattern of a diatonic scale.

In [38]:
// create a major(ionian) key
let e_maj = Key::major(Pitch::E);

e_maj

Key { tonic: E, mode: Ionian }

In [39]:
// The key of Emaj has 4 sharps
e_maj.sharps()

4

In [40]:
// Those sharps are ...
e_maj.alterations()

[FSharp, CSharp, GSharp, DSharp]

In [41]:
// if the key is major, this will return the relative minor (aeolian)
// if minor, will return relative major
// other diatonic modes will return None, which is why this returns option
e_maj.relative()

Some(Key { tonic: CSharp, mode: Aeolian })

In [42]:
// what accidental does G have in E maj?
e_maj.accidental_of(Letter::G)

Sharp

In [43]:
// (same thing) You can also use a key to turn a letter to a pitch in a key.
Letter::G.to_pitch_in_key(e_maj)

GSharp

You can also create a key in other ways.

In [44]:
// .. What key has 6 sharps?
Key::from_sharps(6, DiatonicMode::MAJOR)

Key { tonic: FSharp, mode: Ionian }

In [45]:
// .. What minor key has Bb as its 6th degree?
Key::from_pitch_degree(Numeral7::VI, Pitch::B_FLAT, DiatonicMode::NATURAL_MINOR)

Key { tonic: D, mode: Aeolian }

In [46]:
// .. What key has Db as its tonic, and has 5 flats?
Key::try_from_sharps_tonic(-5, Pitch::D_FLAT)

Some(Key { tonic: DFlat, mode: Ionian })

In [47]:
// That method is useful since keys on other diatonic modes are allowed
Key::try_from_sharps_tonic(-2, Pitch::A)

Some(Key { tonic: A, mode: Locrian })

You can also get a scale from a key. Scales are still a work in progress.

In [48]:
e_maj.scale()

RootedSizedScale { root: E, scale: TypedScale { mode: Ionian } }

In [49]:
e_maj.scale().build_default()

[E, FSharp, GSharp, A, B, CSharp, DSharp]

## Scales
A sequence of octave-repeating* pitches following a specific pattern of intervals. Scales have gone through *two* implementations already, and are still a work in progress.

**Types**

The current impl of scales has three types of scales: an exact scale, like the major scale; a typed scale, like a diatonic scale; and a dynamic scale, which can represent any type of scale.
These different kinds of scales offer type safety, but make the API harder to use, which is why they will be redesigned in the future.

In [50]:
// this is a unit struct which implements ExactScale<7, Scale= DiatonicScaleDef>
MajorScale.build_from(Pitch::D)

[D, E, FSharp, G, A, B, CSharp]

In [51]:
// for a minor scale
// this type is flexible over the mode of the scale
let minor_scale: TypedScale<DiatonicScaleDef, 7> = DiatonicScale::new(DiatonicMode::NATURAL_MINOR);

minor_scale

TypedScale { mode: Aeolian }

In [52]:
minor_scale.build_from(Pitch::G)

[G, A, BFlat, C, D, EFlat, F]

In [53]:
let maj2 = Interval::MAJOR_SECOND;
let m2 = Interval::MINOR_SECOND;
let maj3 = Interval::MAJOR_THIRD;

// finally, there's dynamic scales, which can be anything
// this is a traditional japanese scale for the koto
// this returns an option, since it checks that the intervals sum to an octave
// non octave repeating scales might be implemented when scales are reworked
let iwato = DynamicScale::new([maj2, m2, maj3, m2, maj3]).expect("should sum to P8"); 

iwato.build_from(Pitch::C)

[C, D, EFlat, G, AFlat]

Other scale types can also be converted into dynamic scales.

In [54]:
MajorScale.to_dyn()

DynamicScale { ivls: [Interval { quality: Major, number: IntervalNumber(2) }, Interval { quality: Major, number: IntervalNumber(2) }, Interval { quality: Minor, number: IntervalNumber(2) }, Interval { quality: Major, number: IntervalNumber(2) }, Interval { quality: Major, number: IntervalNumber(2) }, Interval { quality: Major, number: IntervalNumber(2) }, Interval { quality: Minor, number: IntervalNumber(2) }] }

Scales by themselves don't come with many methods, mainly just `.build_from()` which creates the scale from a starting point, and `.interval_between_degrees()`.

In [55]:
// this method uses the numeral type, which ensures the degree is in range.
let p4 = MajorScale.interval_between_degrees(Numeral7::II, Numeral7::V);

p4.to_string()

"P4"

If you give your scale a root, you can do more with it.

In [56]:
let d_maj: RootedSizedScale<Note, 7, MajorScale> = RootedSizedScale { root: Note::new(Pitch::D, 4), scale: MajorScale };

d_maj.build_default().map(|n| n.to_string())

["D4", "E4", "F♯4", "G4", "A4", "B4", "C♯5"]

In [57]:
// build a scale from a starting point up until an end
let scale = d_maj.build(Note::new(Pitch::F, 2), Note::new(Pitch::A, 4));

// debug repr can be verbose, so converting to string
scale.into_iter().map(|n| n.to_string()).collect::<Vec<_>>()

["F♯2", "G2", "A2", "B2", "C♯3", "D3", "E3", "F♯3", "G3", "A3", "B3", "C♯4", "D4", "E4", "F♯4", "G4", "A4"]

In [58]:
// in dmin, an F requires applying a flat to the 3rd degree
// will return None if the letter doesn't exist in the scale
d_maj.get_scale_degree_and_accidental(Note::new(Pitch::F, 4))

Some((3, Flat))

In [59]:
// what's the next note in d min after G#?
d_maj.next_in_scale_after(Note::new(Pitch::G_SHARP, 4))

Note { pitch: A, octave: 4 }

In [60]:
// you can also transpose a scale, which internally transposes its root
let f_maj: RootedSizedScale<Note, 7, MajorScale> = d_maj.transpose(Interval::MINOR_THIRD);

f_maj

RootedSizedScale { root: Note { pitch: F, octave: 4 }, scale: MajorScale }

In [61]:
// you can also get notes at specific scale degrees using .get()
// this takes a numeral type, ensuring the degree is in range
f_maj.get(Numeral7::II)

Note { pitch: G, octave: 4 }

In [62]:
// you can make attempt to make a numeral, if needed
f_maj.get(Numeral7::from_repr(6).expect("6 is a valid scale degree for a heptatonic scale"))

Note { pitch: D, octave: 5 }